<a href="https://colab.research.google.com/github/leah-apking/homework/blob/main/PKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np

In [7]:
pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-dnc343nu
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-dnc343nu
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done


In [8]:
# download the english model
!python -m spacy download en_core_web_sm

2023-10-19 18:42:45.606762: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 18:42:46.793158: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
month='09'
nps_path = f"{month}.ClientNPSSurveyFeedback.csv"
nps_data = pd.read_csv(nps_path)

In [10]:
nps = nps_data[['FullName', 'LoanNumber', 'CompanyName', 'NetPromoterLabel', 'Rating',
       'Comments', 'PerformedBy']]
nps1 = nps.rename(columns={'FullName':'ClientName', 'PerformedBy':'TeamMemberName'})
nps1["Comment"] = nps1.Comments.map(lambda x: 'False' if x == '0' else 'True')

In [11]:
nps2 = nps1.loc[nps1['Comment'] == 'True']
comments = nps2[['LoanNumber', 'Comments']]
comments1 = comments.dropna()

In [12]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_comments = comments.loc[comments['WordCount'] > 10]

<ipython-input-12-a2fc49e891b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


In [ ]:
import pke


ranklist = []
phraselist= []
scorelist = []

def pke_extract(text):
  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases = extractor.get_n_best(n=10, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases):

    # append to list or save to df; rank, phrase and score
    # return newlist.append("rank {}: {} ({})".format(i, candidate, score))
    ranklist.append(i)
    phraselist.append(candidate)
    scorelist.append(score)

for n in range(len(nps_comments)):
    text = nps_comments.iloc[n,1]
    pke_extract(text)

In [15]:
from transformers import pipeline

sentimentlist = []

def sentiment(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentiments = sentiment_pipeline(data)

  # sentimentlist.append(sentiment_pipeline(data))
  for s, (label, score) in enumerate(sentiments):
    sentimentlist.append(label)
    sentimentlist.append(score)

for n in range(len(nps_comments)):
    text = nps_comments.iloc[n,1]
    sentiment(text)

In [28]:
import pke
from transformers import pipeline

sentimentlist = []

ranklist = []
phraselist= []
scorelist = []

def sentiment(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentimentlist.append(sentiment_pipeline(data))

def pke_extract(text):
  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases = extractor.get_n_best(n=10, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases):

    # append to list or save to df; rank, phrase and score
    # return newlist.append("rank {}: {} ({})".format(i, candidate, score))
    ranklist.append(i)
    phraselist.append(candidate)
    scorelist.append(score)

for n in range(len(nps_comments)):
    text = nps_comments.iloc[n,1]
    sentiment(text)
    pke_extract(text)

In [ ]:
sentiment_df = pd.DataFrame(sentimentlist)

def divide_chunks(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 5

phrases = list(divide_chunks(phraselist, n))
columns = ['1st', '2nd', '3rd', '4th', '5th']
df = pd.DataFrame(phrases,columns=columns)

In [48]:
def divide_chunks(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 5

phrases = list(divide_chunks(phraselist, n))
columns = ['1st', '2nd', '3rd', '4th', '5th']
df = pd.DataFrame(phrases,columns=columns)
df

,1st,2nd,3rd,4th,5th
0,uwm,closing team,awesome,easy,process
1,processor,phone,morning,christina plante,jackie berry
2,awesome,hiccups,couple,able,gerry nelson
3,kyle smith,anthony brikho,nathan smedes,garett hess,stephanie ingles
4,jonah rex,life,tbd review,client,conditions
...,...,...,...,...,...
335,life,tbd review,client,conditions,succinct
336,gentleman,prompt,briana kent,file,day
337,questions,loan submission,closing team,help,cortez
338,business,juan herrera-garcia,victoria wells,borrower,ability


In [36]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.5 MB/s eta 0:00:00


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [37]:
from transformers import pipeline

In [47]:
from transformers import pipeline

sentimentlist = []

def sentiment(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentimentlist.append(sentiment_pipeline(data))

for n in range(len(nps_comments)):
    text = nps_comments.iloc[n,1]
    sentiment(text)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-englis

In [49]:
sentiment_df = pd.DataFrame(sentimentlist)
sentiment_df

,0
0,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."
1,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."
2,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."
3,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."
4,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."
...,...
260,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."
261,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."
262,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."
263,"{'label': 'NEGATIVE', 'score': 0.9997116923332..."


In [44]:
sentiment_pipeline = pipeline("sentiment-analysis")
data = nps_comments.iloc[235,1]
something = sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


TypeError: ignored

In [50]:
something

[{'label': 'NEGATIVE', 'score': 0.9997116923332214}]

In [51]:
[{'col1': 'value1', 'col2': 'value2'}]

TypeError: ignored

In [45]:
new_df = pd.DataFrame(something)
new_df

,label,score
0,NEGATIVE,0.999712
